# Goal Module

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.ioff()
from IPython.display import HTML

### Goal Norm Module

In [3]:
import mantrap
import numpy as np
import torch

env = mantrap.environment.PotentialFieldEnvironment(ego_type=mantrap.agents.DoubleIntegratorDTAgent, 
                                                    ego_position=torch.tensor([-5, 0]), 
                                                    ego_velocity=torch.ones(2))

modules = [mantrap.modules.GoalNormModule]
solver = mantrap.solver.SGradSolver(env, goal=torch.tensor([5, 0]),t_planning=5, modules=modules, is_logging=True)
solver.solve(time_steps=10)

HTML(solver.visualize_scenes(legend=True))

### Goal Sum vs Goal Mean Comparison

In [23]:
import collections

import mantrap
import numpy as np
import torch

from mantrap.constants import LK_OVERALL, LT_OBJECTIVE

N = 100
module_dict = {"goal_sum": mantrap.modules.baselines.GoalSumModule, 
               "goal_norm": mantrap.modules.GoalNormModule}

env = mantrap.environment.PotentialFieldEnvironment(ego_type=mantrap.agents.DoubleIntegratorDTAgent, 
                                                    ego_position=torch.zeros(2), 
                                                    ego_velocity=torch.ones(2))

log_dict = collections.defaultdict(list)
for _ in range(N):
    goal = torch.rand(2)
    goal[0] = env.x_axis[0] + (env.x_axis[1] - env.x_axis[0]) * goal[0]
    goal[1] = env.y_axis[0] + (env.y_axis[1] - env.y_axis[0]) * goal[1]
    
    # Solve using each objective and query for the objective loggings.
    for module_key, module in module_dict.items():
        solver = mantrap.solver.SGradSolver(env, goal=goal, modules=[module], is_logging=True)
        solver.solve(time_steps=10)
        obj_log = solver.log_query(key_type=LT_OBJECTIVE, key=LK_OVERALL, last=True)
        log_dict[module_key].append(obj_log)

In [24]:
# Number of iterations.
num_iters = {key: np.mean([obj.numel() for obj in objectives]) for key, objectives in log_dict.items()}
num_iters

{'goal_sum': 8.99, 'goal_norm': 9.02}

In [33]:
# 95% decrease of objective value.
decay = {}
for key, objectives in log_dict.items():
    decay_key = []
    for objective in objectives:
        objective_np = objective.detach().numpy()
        obj_decayed = objective_np[0] - 0.95 * (objective_np[0] - objective_np[-1])
        decay_key.append(np.argmax(objective_np < obj_decayed))
    decay[key] = np.mean(decay_key)
decay

{'goal_sum': 6.2, 'goal_norm': 6.75}